In [1]:
#Run once per session
!pip install fastai wwf -q --upgrade

     |████████████████████████████████| 548 kB 7.6 MB/s 
     |████████████████████████████████| 163 kB 24.3 MB/s 


In [2]:
from fastai.tabular.all import *

In [4]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [29]:
x_vars = list(train_df.columns.values[1:])
y_vars = train_df.columns.values[0]

In [30]:
# validation split
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

In [32]:
to = TabularPandas(train_df, procs=[Normalize],
                   cont_names = x_vars,
                   y_names = y_vars,
                   splits = splits)

In [33]:
dls = to.dataloaders(bs=64)

In [90]:
learn = tabular_learner(dls, metrics=mae)
# test df
test_dl = learn.dls.test_dl(test_df.drop(['q'], axis=1))

In [91]:
learn.fit_one_cycle(100)

epoch,train_loss,valid_loss,mae,time
0,0.995548,0.913485,0.948996,00:00
1,0.965956,0.871929,0.923754,00:00
2,0.949602,0.840214,0.903755,00:00
3,0.937090,0.819892,0.891841,00:00
4,0.926149,0.820909,0.891245,00:00
5,0.917720,0.813642,0.887522,00:00
6,0.909031,0.773673,0.869638,00:00
7,0.901952,0.782953,0.869187,00:00
8,0.893988,0.819002,0.887865,00:00
9,0.885680,0.803538,0.884042,00:00


In [92]:
from sklearn.metrics import mean_absolute_error
y_test_pred = learn.get_preds(dl=test_dl)[0].numpy()
mae_test = mean_absolute_error(y_test_pred, test_df.q.values)
mae_test

0.027339877864964283

In [94]:
learn.model

TabularModel(
  (embeds): ModuleList()
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(276, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=276, out_features=200, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): LinBnDrop(
      (0): Linear(in_features=200, out_features=100, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=100, out_features=1, bias=True)
    )
  )
)

In [95]:
learn.summary()

TabularModel (Input shape: 64 x 0)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 276            
BatchNorm1d                               552        True      
____________________________________________________________________________
                     64 x 200            
Linear                                    55200      True      
ReLU                                                           
BatchNorm1d                               400        True      
____________________________________________________________________________
                     64 x 100            
Linear                                    20000      True      
ReLU                                                           
BatchNorm1d                               200        True      
____________________________________________________________________________
                     64 x 1              
Linear                                    101        T